In [19]:
# https://huggingface.co/docs/trl/en/sft_trainer#format-your-input-prompts
# https://huggingface.co/docs/trl/en/sft_trainer#supervised-fine-tuning-trainer
def format_reddit_post(sample):
    answer = sample["answer\r"]
    question = sample["question"]
    conversation = {
        "messages": [
            {"role": "system", "content": "You help answer questions for UT Austin students"},
            {"role": "user", "content": question },
            {"role": "assistant", "content": answer}

        ]
    }

    return conversation

from datasets import load_dataset, Dataset
import pandas as pd

# load the processed dataset
base_path = "Bevo-Bud-The-GPT/data"
# train_data = load_dataset("csv", data_files=f"{base_path}/train-v1.csv")
# test_data = load_dataset("json", data_files=f"{base_path}/train-v1.json")

train_df = pd.read_csv(f"{base_path}/train-v1.csv", lineterminator='\n')
train_df.dropna(inplace=True)

dataset_hf = Dataset.from_pandas(train_df)
# dataset_hf['answer\r']
train_data = dataset_hf.map(format_reddit_post, batched=False, remove_columns=["question", "answer\r"])

Map:   0%|          | 0/5301 [00:00<?, ? examples/s]

In [20]:
train_data['messages'][0]

[{'content': 'You help answer questions for UT Austin students',
  'role': 'system'},
 {'content': 'Why are the ice cream machines at J2 always broken? ',
  'role': 'user'},
 {'content': 'I’m not sure if it’s still the same, but a few years ago I worked at J2 and the lady that was in charge of the desserts area always complained about having to clean the machines. I’m pretty sure she just turned them off on days she didn’t feel like cleaning them. \r',
  'role': 'assistant'}]

In [32]:
from transformers import AutoTokenizer, TrainingArguments
model_checkpoint = "distilbert/distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, max_length=1102)
tokenizer.pad_token = tokenizer.eos_token
version = "1.1"
print("Saving as verion: ", version)

Saving as verion:  1.1


In [34]:
from trl import SFTTrainer 
from peft import LoraConfig

training_args = {
    # "per_device_train_batch_size": 2,
    # "per_device_eval_batch_size": 2,
    # "gradient_accumulation_steps": 4,
    "num_train_epochs": 10,
    # "logging_steps": 1,
    # "save_steps": 100,
    "seed": 42,
    "output_dir": f"{base_path}/Bevo-Bud",
    # "push_to_hub": True,
    "overwrite_output_dir": True
}

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
training_args = TrainingArguments(**training_args)
trainer = SFTTrainer(
            model=model_checkpoint,
            train_dataset=train_data,
            tokenizer=tokenizer,
            packing=True,
            peft_config=peft_config,
            max_seq_length=1024,
            args=training_args,
        )

trainer.train()

Generating train split: 0 examples [00:00, ? examples/s]

Step,Training Loss
500,4.034800


TrainOutput(global_step=890, training_loss=3.9344483793451546, metrics={'train_runtime': 15913.9084, 'train_samples_per_second': 0.445, 'train_steps_per_second': 0.056, 'total_flos': 1862809524633600.0, 'train_loss': 3.9344483793451546, 'epoch': 10.0})

In [35]:
trainer.save_model(f"{base_path}/model")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1714030355.82d7450fbbdd.30.6:   0%|          | 0.00/5.57k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

### Testing

In [36]:
from transformers import pipeline

generator = pipeline('text-generation', model=f"{base_path}/model", tokenizer=tokenizer)
# generator = pipeline('text-generation', model=f"{base_path}/Bevo-Budv{version}", tokenizer=tokenizer)

In [40]:

generator("{'content': 'You help answer questions for UT Austin students','role': 'system'},{'content': 'Can I still graduate if my GPA is below the minimum?','role': 'user'}")

[{'generated_text': "{'content': 'You help answer questions for UT Austin students','role': 'system'},{'content': 'Can I still graduate if my GPA is below the minimum?','role': 'user'}},{'content': '"}]

## Uploading Model To Hub

In [82]:
from huggingface_hub import notebook_login
notebook_login()
trainer.push_to_hub(f"Bevo-Budv", )

events.out.tfevents.1713985198.82d7450fbbdd.5942.0:   0%|          | 0.00/5.00k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1713985235.82d7450fbbdd.5942.1:   0%|          | 0.00/5.57k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kelechie/tmp_trainer/commit/a5daaa92c8176af9a9d307f73b97b613e83db901', commit_message='Bevo-Budv1.0', commit_description='', oid='a5daaa92c8176af9a9d307f73b97b613e83db901', pr_url=None, pr_revision=None, pr_num=None)